# Statistics
This notebook provides an overview of the functions provided by the elephant `statistics` module.


In [ ]:
import sys
#sys.path.insert(0,"../../")

from elephant.statistics import *

In [ ]:
!pwd

In [ ]:
import elephant
print(elephant.__file__)

In [ ]:
for x in sys.path:
    print(x)

In [ ]:
!ls /home/docs/checkouts/readthedocs.org/user_builds/elephant-docs-development/conda/enh-module_doc/lib/python3.6/site-packages

In [ ]:
!ls /home/docs/checkouts/readthedocs.org/user_builds/elephant-docs-development/conda/enh-module_doc/lib/python3.7/site-packages

All measures presented here require one or two spiketrains as input. We first generate two spiketrains using the elephant `spike_train_generation` module.

In [ ]:
from elephant.spike_train_generation import homogeneous_poisson_process, homogeneous_gamma_process
from numpy import array
from quantities import ms, Hz
homogeneous_poisson_process
spiketrain1 = homogeneous_poisson_process(10*Hz, t_start=array(0.) * ms, t_stop=array(10000.) * ms)
spiketrain2 = homogeneous_gamma_process(3, 10*Hz, t_start=array(0.) * ms, t_stop=array(10000.) * ms)

## Rate estimation

Elephant offers three approaches for estimating the underlying rate of a spike train. These functions all take a spike train as input and provide an estimate of the rate using different approaches.

### Mean firing rate

The simplest approach is to assume a stationary firing rate and only use the total number of spikes and the duration of the spike train to calculate the average number of spikes per time unit. This results in a single value for a given spiketrain.

In [ ]:
mean_firing_rate(spiketrain1)

Additionally, the period within the spike train during which to estimate the firing rate can be further limited using the `t_start` and `t_stop` keyword arguments. Here, we limit the firing rate estimation to the first second of the spiketrain.

In [ ]:
mean_firing_rate(spiketrain1, t_start=0*ms, t_stop=1000*ms)

In some (rare) cases multiple spiketrains can be represented in multidimensional arrays when they contain the same number of spikes. In such cases, the mean firing rate can be calculated for multiple spiketrains at once by specifying the axis the along which to calculate the firing rate. By default, if no axis is specified, all spiketrains are pooled together before estimating the firing rate.

In [ ]:
multi_spiketrains = array([[1,2,3],[4,5,6],[7,8,9]])*ms
mean_firing_rate(multi_spiketrains, axis=0, t_start=0*ms, t_stop=5*ms)

### Time histogram
The time histogram is a time resolved way of the firing rate estimation. Here, the spiketrains are binned and either the count or the mean count or the rate of the spiketrains is returned, depending on the `output` parameter. The result is a count (mean count/rate value) for each of the bins evaluated. This is represented as a neo `AnalogSignal` object with the corresponding sampling rate and the count (mean count/rate) values as data.

Here, we compute the counts of spikes in bins of 500 millisecond width.

In [ ]:
histogram_count = time_histogram([spiketrain1], 500*ms)

This returns an `AnalogSignal` where each sample is the count of spike in the bin corresponding to the sampling time.

In [ ]:
print('times:', histogram_count.times)
print('counts:', histogram_count[:,0].T)

Alternatively, `time_histogram` can also normalize the resulting array to represent a mean counts or a rate.

In [ ]:
histogram_rate = time_histogram([spiketrain1], 500*ms, output='rate')

In [ ]:
print('times:', histogram_rate.times)
print('rate:', histogram_rate[:,0].T)

Additionally, `time_histogram` can be limited to a shorter time period by using the keyword arguments `t_start` and `t_stop`, as described for `mean_firing_rate`.

### Instantaneous rate

The instantaneous rate is, similar to the time histogram (see above), providing a continuous estimate of the underlying firing rate of a spike train. Here, the firing rate is estimated as a convolution of the spiketrain with a firing rate kernel, representing the contribution of a single spike to the firing rate. In contrast to the time histogram, the instantaneous rate provides a smooth firing rate estimate as it does not rely on binning of the spiketrain.

Estimation of the instantaneous rate requires a sampling period on which the firing rate is estimated. Here we use a sampling rate of 50 millisecond.

In [ ]:
inst_rate = instantaneous_rate(spiketrain1, 50*ms)

The resulting rate estimate is again an `AnalogSignal` with the sampling rate defined above.

In [ ]:
print('sampling rate:', inst_rate.sampling_rate)
print('times:', inst_rate.times)
print('instantaneous rate:', inst_rate[:,0].T)

Additionally, also the type of kernel used for the convolution can be specified via the `kernel` keyword argument. E.g. to use an gaussian kernel, we specify the following.

In [ ]:
from elephant.kernels import GaussianKernel
instantaneous_rate(spiketrain1, 20*ms, kernel=GaussianKernel(200*ms))

To compare all three methods of firing rate estimation, we visualize the results of all methods in a common plot.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))

# plotting original spiketrain
plt.plot(spiketrain1, [0]*len(spiketrain1), 'r', marker=2, ms=25, markeredgewidth=5, lw=0, label='poisson spike times')

# mean firing rate
plt.hlines(mean_firing_rate(spiketrain1), xmin=spiketrain1.t_start, xmax=spiketrain1.t_stop, label='mean_firing_rate')

# time histogram
plt.bar(histogram_rate.times, histogram_rate.magnitude.flatten(), width=histogram_rate.sampling_period, align='edge', alpha=0.3, label='time histogram (rate)')

# instantaneous rate
plt.plot(inst_rate.times.rescale(ms), inst_rate.rescale(histogram_rate.dimensionality).magnitude.flatten(), label='instantaneous rate')

# axis labels and legend
plt.xlabel('time [{}]'.format(spiketrain1.times.dimensionality.latex))
plt.ylabel('firing rate [{}]'.format(histogram_rate.dimensionality.latex))
plt.xlim(spiketrain1.t_start, spiketrain1.t_stop)
plt.legend()
plt.show()

# TODO

## Spike interval statistics

## Statistics across spike trains